In [1]:
%qtconsole

In [2]:
import numpy as np
import tensorflow as tf
import spektral

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GeneralConv
from spektral.layers import GlobalSumPool
from spektral.data import DisjointLoader, Dataset
from spektral.datasets import TUDataset
from spektral.datasets.graphsage import Reddit
from spektral.layers import GraphSageConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from spektral.data import BatchLoader
from tensorflow.keras.layers import Layer, Dense, Activation, Input, Lambda, Concatenate, Dropout, ReLU, Reshape,Flatten

from tensorflow.keras.optimizers import SGD, Adam

/home/nacho/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
#Model creation
#Idea from joewilaj/nbaGNNs

class To_Sparse(Layer):

    def __init__(self,):
        super(To_Sparse,self).__init__()

    def call(self,inputs):

        sparse_t = tf.sparse.from_dense(inputs[0], name = 'adj_mat_sp')

        return sparse_t



class node_pair_xt(Layer):

    def __init__(self):
        super(node_pair_xt,self).__init__()

    def call(self,inputs):
        
        extracted = tf.gather(inputs[1],inputs[0], axis=1)

        return extracted


def LinkPredictor(nodes_n,features_n,ch_gnn1,ch_gnn2,drop_rate):

    #Inputs
    raw_adj = Input(shape=(nodes_n,nodes_n),name='adj_mtx')
    adjmtx = To_Sparse()(raw_adj)
    nodefeat= Input(shape=(nodes_n, features_n), name='feature_mtx')
    index_train = Input(shape=(2,),dtype = tf.int64, name='index_pair_nodes')
    
    #Layers
    conv1 = spektral.layers.GeneralConv(channels= ch_gnn1, name='gcn1')([nodefeat,adjmtx])
    conv2 = spektral.layers.GeneralConv(channels= ch_gnn2, name='gcn2')([conv1,adjmtx])
    extracted = node_pair_xt()([index_train,conv2])
    rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
    
    dense = Dense(64)(rshp)
    dense = Dropout(drop_rate)(dense)
    dense = Dense(12)(dense)
    dense = Dropout(drop_rate)(dense)
    output_layer = Dense(2, 'softmax')(dense)
    
    #Model
    model = Model(inputs= [raw_adj, nodefeat, index_train ], outputs = output_layer)
       
    return model
   

In [4]:
link_pred = LinkPredictor(nodes_n=20, features_n=12, ch_gnn1=64, ch_gnn2=128, drop_rate=0.3)
link_pred.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 adj_mtx (InputLayer)           [(None, 20, 20)]     0           []                               
                                                                                                  
 feature_mtx (InputLayer)       [(None, 20, 12)]     0           []                               
                                                                                                  
 to__sparse (To_Sparse)         (20, 20)             0           ['adj_mtx[0][0]']                
                                                                                                  
 gcn1 (GeneralConv)             (None, 20, 64)       2368        ['feature_mtx[0][0]',            
                                                                  'to__sparse[0][0]']         

In [5]:
opt = Adam(learning_rate=0.001)

In [6]:
def LPred_training_set(input_matrix, feat_vec, batch_size):
    
    index_train = np.zeros((batch_size,2),dtype = int)
    y_train = np.zeros((batch_size,2),dtype = float)
    adjmtx = np.zeros((batch_size, InputMatrix.shape[0],InputMatrix.shape[1]),dtype = int)
    
    #construct matrix
    
    for index in range(batch_size):
        #mask 1 edge
        #construct y (half time right/ half time false)
        
        
    
    return index_train,adjmtx,nodefeat,y_train
    

IndentationError: expected an indented block (<ipython-input-6-62145db75698>, line 15)

In [7]:
#End2end test on random matrices

Ad_input = np.random.rand(10,20,20)
Ad_input = Ad_input.round()

Feat_input = np.random.rand(10,20,12)

index_train = np.random.randint(1,20, size=(10, 2))

y=np.random.rand(10)
y=y.round()

y =tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32')

In [8]:
#print(Ad_input[1])

no_links =np.where(Ad_input[1] == 0)
links =np.where(Ad_input[1] == 1)

for item in range(len(no_links[0])):
    if(item == 0):
        index_no_link = [no_links[0][item], no_links[1][item]]
    else:
        #print(index[0][item], index[1][item])
        index_no_link = np.vstack([index_no_link,  [no_links[0][item], no_links[1][item]] ])


for item in range(len(links[0])):
    if(item == 0):
        index_link = [links[0][item], links[1][item]]
    else:
        #print(index[0][item], index[1][item])
        index_link = np.vstack([index_link,  [links[0][item], links[1][item]] ])


In [9]:

link_pred.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])

link_pred.fit([Ad_input,Feat_input,index_train],y, epochs = 200,batch_size = 1)


Epoch 1/200
10/10 [==============================] - 2s 4ms/step - loss: 4.5153 - accuracy: 0.4000 
Epoch 2/200
10/10 [==============================] - 0s 5ms/step - loss: 4.7244 - accuracy: 0.6000
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.4200 - accuracy: 0.7000
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9856 - accuracy: 0.8000
Epoch 5/200
10/10 [==============================] - 0s 6ms/step - loss: 1.5050 - accuracy: 0.5000
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 4.6996 - accuracy: 0.7000
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 2.9657 - accuracy: 0.6000
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.7547 - accuracy: 0.7000
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 0.2657 - accuracy: 1.0000
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 0.6295 - accuracy: 0.9000
Epoch 11

In [ ]:
class DataGeneratorLinkPredictor(keras.utils.Sequence):
    
    def __init__(self, adj_mtx, edg_lst, zer_lst, nod_emb, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.adj_mtx = adj_mtx
        self.edg_lst = edj_lst
        self.nod_emb = nod_emb
        self.shuffle = shuffle
        self.edg_list = edg_lst
        self.zer_list = zer_lst
        self.zeros  = zer_lst.shape[0]
        self.ones  = edg_lst.shape[0]
        self.on_epoch_end()
        

    def __len__(self):
        if(self.ones >= self.zeros):
            items_per_batch = int(np.floor(self.zeros / self.batch_size))
        else:
            items_per_batch = int(np.floor(self.ones / self.batch_size))

        return items_per_batch
            

    def __getitem__(self, index):
        
        index0 = self.indexes_0[index*self.batch_size:(index+1)*self.batch_size,]
        index1 = self.indexes_1[index*self.batch_size:(index+1)*self.batch_size,]

        # Generate data
        x_adj, y = self.__data_generation(index1,index0)

        return x_adj, y

    def on_epoch_end(self):
        ione = np.arange(self.ones)
        izero = np.arange(self.zeros)
        
        
        if self.shuffle == True:
            np.random.shuffle(ione)
            np.random.shuffle(izero)
        
        self.indexes_0 = self.zer_list[izero,]
        self.indexes_1 = self.edg_list[ione,]

    def __data_generation(self, index1, index0):

        for i in range(index0.shape[0]):
            # Store sample
            X = self.adj_mtx
            X[index0[i,]]=1
            X = np.expand_dims(X, axis=0)
            Y = 0
            if(i == 0):
                x_out = X
                y_out=0
            else:
                x_out = np.concatenate([x_out,X], axis=0)
                y_out = np.concatenat([y_out,Y])
                
        for i in range(index1.shape[0]):
            # Store sample
            X = self.adj_mtx
            X[index1[i,]]=0
            X = np.expand_dims(X, axis=0)
            Y = 1
            x_out = np.concatenate([x_out,X], axis=0)
            y_out = np.concatenat([y_out,Y])

            #shuffle
            x_out = x_out[np.random.shuffle(np.arange(x_out.shape[0])),,]


        return x_out, self.nod_emb, keras.utils.to_categorical(y_out, num_classes=self.n_classes)